## Data from:
https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A1#bands

In [72]:
import pandas as pd
from datetime import timedelta

 # Read CSV

In [73]:
weekly_temperature = pd.read_csv('Data/ModisDailyTemperature.csv')
weekly_temperature.rename(columns={'system:time_start' : 'Date', 'LST_Day_1km' : 'temperature'}, inplace=True)
weekly_temperature['Date'] = pd.to_datetime(weekly_temperature['Date'])
weekly_temperature

,Date,temperature
0,2006-12-31,NaN
1,2007-01-01,23.960
2,2007-01-02,NaN
3,2007-01-03,15.337
4,2007-01-04,NaN
...,...,...
4732,2019-12-26,25.203
4733,2019-12-27,NaN
4734,2019-12-28,22.245
4735,2019-12-29,21.166


### Fill NaN with last value

In [74]:
weekly_temperature['ffill'] = weekly_temperature.temperature.fillna(method="ffill")
# Fill the NaN of firsth row
weekly_temperature['ffill'] = weekly_temperature['ffill'].fillna(method="backfill")
weekly_temperature

,Date,temperature,ffill
0,2006-12-31,NaN,23.960
1,2007-01-01,23.960,23.960
2,2007-01-02,NaN,23.960
3,2007-01-03,15.337,15.337
4,2007-01-04,NaN,15.337
...,...,...,...
4732,2019-12-26,25.203,25.203
4733,2019-12-27,NaN,25.203
4734,2019-12-28,22.245,22.245
4735,2019-12-29,21.166,21.166


### Fill NaN with next value

In [75]:
weekly_temperature['backfill'] = weekly_temperature.temperature.fillna(method="backfill")
# Fill the NaN of last row
weekly_temperature['backfill'] = weekly_temperature['backfill'].fillna(method="ffill")
weekly_temperature

,Date,temperature,ffill,backfill
0,2006-12-31,NaN,23.960,23.960
1,2007-01-01,23.960,23.960,23.960
2,2007-01-02,NaN,23.960,15.337
3,2007-01-03,15.337,15.337,15.337
4,2007-01-04,NaN,15.337,20.931
...,...,...,...,...
4732,2019-12-26,25.203,25.203,25.203
4733,2019-12-27,NaN,25.203,22.245
4734,2019-12-28,22.245,22.245,22.245
4735,2019-12-29,21.166,21.166,21.166


## Create a new temperature column with NaN as mean of last and next temperature

In [76]:
weekly_temperature['temperature'] = (weekly_temperature['backfill'] + weekly_temperature['ffill'])/2
weekly_temperature

,Date,temperature,ffill,backfill
0,2006-12-31,23.9600,23.960,23.960
1,2007-01-01,23.9600,23.960,23.960
2,2007-01-02,19.6485,23.960,15.337
3,2007-01-03,15.3370,15.337,15.337
4,2007-01-04,18.1340,15.337,20.931
...,...,...,...,...
4732,2019-12-26,25.2030,25.203,25.203
4733,2019-12-27,23.7240,25.203,22.245
4734,2019-12-28,22.2450,22.245,22.245
4735,2019-12-29,21.1660,21.166,21.166


# Calculate the mean by week

In [77]:
# https://towardsdatascience.com/daily-to-weekly-pandas-c0557b12f052
new_weekly_temperature = weekly_temperature.resample('W', label='left', closed = 'left', on='Date').mean()
new_weekly_temperature['LastDayWeek'] = new_weekly_temperature.index
new_weekly_temperature['LastDayWeek'] = new_weekly_temperature['LastDayWeek'] + timedelta(days=6)

In [78]:
new_weekly_temperature

,temperature,ffill,backfill,LastDayWeek
Date,,,,
2006-12-31,19.615357,19.032571,20.198143,2007-01-06
2007-01-07,22.234000,22.469571,21.998429,2007-01-13
2007-01-14,22.268857,22.450286,22.087429,2007-01-20
2007-01-21,23.546000,23.642143,23.449857,2007-01-27
2007-01-28,25.353071,25.135000,25.571143,2007-02-03
...,...,...,...,...
2019-12-01,23.005214,23.675286,22.335143,2019-12-07
2019-12-08,23.116214,23.111000,23.121429,2019-12-14
2019-12-15,21.851143,21.728857,21.973429,2019-12-21


In [79]:
# Dataframe to CSV
new_weekly_temperature.to_csv(r'new_weekly_temperature.csv')